<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">


# GIS

This notebook shows how to export model data so it can be viewed in GIS (QGIS or ArcMAP).

### Contents<a name="TOC"></a>
1. [Model types](#modeltypes)
2. [Export vector data](#vectordata)
3. [Export griddata](#rasterdata)
4. [Add symbology (QGIS)](#symbology)

In [ ]:
import nlmod
import os
import flopy
import numpy as np
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

import logging
from IPython.display import FileLink, FileLinks

In [ ]:
print(f'nlmod version: {nlmod.__version__}')

# toon informatie bij het aanroepen van functies
logging.basicConfig(level=logging.INFO)

### [1. Model types](#TOC)<a name="modeltypes"></a>



#### structured grid



In [ ]:
model_ws = 'model1'
model_name = 'IJmuiden'

In [ ]:
model_ds_struc = xr.load_dataset(os.path.join(model_ws, 'cache', 'full_model_ds.nc'), mask_and_scale=False)
model_ds_struc

In [ ]:
# create gisdir
gisdir_struc = os.path.join(model_ws, 'gis')
if not os.path.exists(gisdir_struc):
    os.mkdir(gisdir_struc)

#### vertex grid



In [ ]:
model_ws = 'model3'
model_name = 'IJm_planeten'

In [ ]:
model_ds_vert = xr.load_dataset(os.path.join(model_ws, 'cache', 'full_model_ds.nc'), mask_and_scale=False)

# get modelgrid
sim = flopy.mf6.MFSimulation.load(sim_name='mfsim.nam', sim_ws=model_ds_vert.model_ws,
                                  load_only=['DISV'])
gwf = sim.get_model(model_ds_vert.model_name)

# get vertices
model_ds_vert['vertices'] = nlmod.mdims.get_vertices(model_ds_vert, modelgrid=gwf.modelgrid)
model_ds_vert

In [ ]:
# create gisdir
gisdir_vert = os.path.join(model_ws, 'gis')
if not os.path.exists(gisdir_vert):
    os.mkdir(gisdir_vert)

### [2. Export vector data](#TOC)<a name="vectordata"></a>



#### structured

In [ ]:
# write model data to a geopackage
fname_geopackage = nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_struc, gisdir=gisdir_struc)

# get download link
FileLink(fname_geopackage, result_html_prefix='klik hier om te downloaden -> ')

In [ ]:
# write model data to multiple shapefiles
fnames = nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_struc, driver='ESRI Shapefile', gisdir=gisdir_struc)

# get download link
FileLinks(gisdir_struc, included_suffixes='.shp')

#### vertex grid

In [ ]:
# write model data to a geopackage
fname_geopackage = nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_vert, gisdir=gisdir_vert)

In [ ]:
# write model data to multiple shapefiles
nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_vert, driver='ESRI Shapefile', gisdir=gisdir_vert)

### [3. Export griddata](#TOC)<a name="rasterdata"></a>

The model data can also be exported as a NetCDF file. For a structured model the standard model dataset (xarray.Dataset) can be written to a netcdf and read using Qgis. For a vertex model you have to convert the model dataset to a certain format before you can write it to a netcdf and read it with Qgis. With the code below we export the vertex model dataset to a netcdf file ('model_qgis.nc') that can be read using Qgis. For more background, see this discussion https://github.com/ArtesiaWater/nlmod/issues/14. 

In [ ]:
fname = os.path.join(model_ds_vert.cachedir,'model_qgis.nc')
out = nlmod.visualise.gis.model_dataset_to_ugrid_nc_file(model_ds_vert.drop_vars('rch_name'), fname)
out

### [4. Add symbology (QGIS)](#TOC)<a name="symbology"></a>

It is always nice to have automatic symbology for your vector data. Some thoughts:
- QGIS can save symbology of a single shapefile in a .qml file
- In QGIS you can add a .qml file to a geopackage thus saving the symbology to a single file.
- You can create a .qml file in QGIS from existing symbology.
- a .qml file is an xml file so theoretically it is possible to modify a .qml file with Python based on the properties of the data.
- It 



Some limitations of the current gis functions:
- when exporting shapefiles to gis, attributes cannot have names longer
than 10 characters. Now the automatic character shortening of fiona is
used. This is not optimal.
- when exporting data variables with dimension time only the mean values
in time are exported in the shapefile to avoid extremely large files.